# Linear Regression Implementation from Scratch

In [6]:
import random
import torch

In [7]:
#Generating the Dataset

def synthetic_data(w, b, num_examples):
  """Generate y = Xw + b + noise"""
  X = torch.normal(0, 1, (num_examples, len(w))) 
  #making normal distribution (mean = 0, std = 16)
  #(num_examples, len(w)) = size -> a sequence of integers defining the shape of the ouput tensor
  y = torch.matmul(X, w) + b
  y += torch.normal(0, 0.01, y.shape)
  return X, y.reshape((-1, 1))
  # -1 -> estimating from the dimention
  # 1 -> confirmed by user
  # y: 1차원의 타켓 값으로 재구성

In [8]:
true_w = torch.tensor([2, -3.4])
true_b = 4.2
features, labels = synthetic_data(true_w, true_b, 1000)

In [9]:
print('features: ', features[0], '\nlabel: ', labels[0])

features:  tensor([-0.4806,  0.2970]) 
label:  tensor([2.2242])


In [10]:
# Reading the dataset

#data_iter: 임의로 선택된 feature과 tag들을 batch size 만큼 리턴
#왜 하나의 샘플을 사용하지 않고 여러 샘플을 사용? -> 최적화를 효율적으로!
def data_iter(batch_size, features, labels):
  num_examples = len(features)
  indices = list(range(num_examples))
  random.shuffle(indices)

  #이 함수 이해 !!!
  for i in range(0, num_examples, batch_size):
    batch_indices = torch.tensor(
        indices[i: min(i + batch_size, num_examples)])
    yield features[batch_indices], labels[batch_indices]

In [11]:
batch_size = 10

for X, y in data_iter(batch_size, features, labels):
  print(X, '\n', y)
  break

tensor([[-2.1558, -1.4840],
        [ 0.6180, -0.2540],
        [-0.4332,  0.2474],
        [ 0.9920, -0.3920],
        [ 0.8425,  0.6082],
        [-0.3384,  0.1283],
        [-0.7787, -0.4093],
        [ 0.0390, -0.4910],
        [ 0.8529, -0.6465],
        [ 0.6911, -0.9827]]) 
 tensor([[4.9387],
        [6.2925],
        [2.4789],
        [7.5097],
        [3.8202],
        [3.0963],
        [4.0277],
        [5.9394],
        [8.1163],
        [8.8953]])


In [12]:
#Initializing Model Parameters

w = torch.normal(0, 0.01, size = (2,1), requires_grad=True)
b = torch.zeros(1, requires_grad=True)
#requires_grad 속성을 True로 설정 = 그 tensor에서 이뤄진 모든 연산들을 추적

In [13]:
#Defining the Model

def linreg(X, w, b):
  """The linear regression model"""
  return torch.matmul(X, w) + b

In [14]:
#Defining the Loss Function

def squared_loss(y_hat, y):
  """Squred loss"""
  return (y_hat - y.reshape(y_hat.shape))**2 /2
  #y의 모양을 y_hat의 모양과 동일하게 바꿈

In [15]:
#Defining the Optimization Algorithm

def sgd(params, lr, batch_size):
  """Minibatch stochastic gradient descent"""
  with torch.no_grad(): 
    #메모리 사용량을 줄이기 위한 것 -> autograd가 requires_grad = True인 Tensor들의 연산 기록을 추적하는 것을 멈출 수 있음
    for param in params:
      param -= lr * param.grad / batch_size
      param.grad.zero_()

In [16]:
#Training

lr = 0.03
num_epochs = 3
net = linreg
loss = squared_loss

In [17]:
for epoch in range(num_epochs):
  for X, y in data_iter(batch_size, features, labels):
    l = loss(net(X, w, b), y)
    l.sum().backward() #역전파 Backward Propagation
    sgd([w, b], lr, batch_size)
  with torch.no_grad():
      train_l = loss(net(features, w, b), labels)
      print(f'epoch {epoch + 1}, loss {float(train_l.mean()): f}')

epoch 1, loss  0.035527
epoch 2, loss  0.000123
epoch 3, loss  0.000048


In [18]:
print(f'error in estimating w: {true_w - w.reshape(true_w.shape)}')
print(f'error in estimating b: {true_b - b}')

error in estimating w: tensor([ 0.0008, -0.0001], grad_fn=<SubBackward0>)
error in estimating b: tensor([0.0002], grad_fn=<RsubBackward1>)
